# Senti-Pred: Pipeline Completo de Análise de Sentimentos
## Autor: Pedro Morato Lahoz

---

### Índice:
1. [Configuração Inicial](#config)
2. [Análise Exploratória (EDA)](#eda)
3. [Pré-processamento](#preprocessing)
4. [Modelagem](#modeling)
5. [Visualizações Comparativas](#visualizations)
6. [Deploy com Docker](#deploy)


---
## 1. Configuração Inicial <a id='config'></a>

(Se estiver usando Colab, execute a célula de setup em seguida.)

In [ ]:
# Setup para Google Colab (executar somente no Colab)
# Detecta ambiente Colab, clona repo e instala dependências.
import sys, os  # sys para detectar módulos (Colab), os para manipulação de caminhos
from pathlib import Path  # Path para manipulação robusta de paths
IN_COLAB = 'google.colab' in sys.modules  # True em Colab
REPO = 'PedroM2626/Senti-Pred'  # repo para clonar se necessário
CLONE_DIR = Path('/content/Senti-Pred')

if IN_COLAB:
    if not CLONE_DIR.exists():
        print('Clonando repositorio para /content/Senti-Pred...')
        get_ipython().system(f'git clone https://github.com/{REPO}.git {CLONE_DIR}')
    %cd /content/Senti-Pred
    print('Instalando dependencias... (pode demorar)')
    # pip install -r requirements.txt para garantir libs corretas
    get_ipython().system('pip install -q -r requirements.txt || true')
else:
    print('Nao detectado Colab; assumindo execucao local. Verifique dependencias manualmente.')

# criar diretórios usados pelo notebook
os.makedirs('data/raw', exist_ok=True)
os.makedirs('reports/visualizacoes', exist_ok=True)
os.makedirs('reports/metrics', exist_ok=True)
os.makedirs('src/models', exist_ok=True)

# baixar recursos NLTK de forma silenciosa (pode já estar instalado)
import nltk  # biblioteca de processamento de linguagem natural
for r in ['punkt','stopwords','wordnet','omw-1.4','averaged_perceptron_tagger']:
    try:
        nltk.download(r, quiet=True)
    except Exception:
        pass

print('Setup concluido. IN_COLAB =', IN_COLAB)

---
## Imports e Configuração Inicial

Aqui importamos todas as bibliotecas necessárias. Cada import tem um comentário curto explicando por que está sendo usado no pipeline.

In [ ]:
# Imports com comentarios explicativos
import os  # manipulação de caminhos, criação de pastas, checagens de arquivos
import re  # regex para limpeza de texto (URLs, menções, caracteres especiais)
import json  # salvar/ler metadados e metricas em formato JSON
import time  # medir tempos de treino/inferencia
import warnings  # silenciar avisos não-críticos durante o notebook
warnings.filterwarnings('ignore')

import joblib  # salvar/carregar pipelines e modelos de forma eficiente
import numpy as np  # operações numéricas e manipulação de arrays
import pandas as pd  # leitura de CSVs e manipulação de dataframes
import matplotlib.pyplot as plt  # plotagem estática de gráficos
import seaborn as sns  # visualizações estatísticas (heatmaps, estilo)
from pathlib import Path  # objetos Path para manipulação de caminhos com segurança
sns.set(style='whitegrid')  # estilo padrão para os plots

nltk.download('punkt_tab')

# Determinacao robusta de BASE_DIR (funciona em scripts e notebooks/Colab)
try:
    BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
except Exception:
    # se __file__ nao existe (notebook), tentamos usar o path clonado no Colab
    if 'IN_COLAB' in globals() and IN_COLAB:
        candidate = Path('/content/Senti-Pred')
        if candidate.exists():
            BASE_DIR = str(candidate.resolve())
        else:
            BASE_DIR = str(Path.cwd())
    else:
        BASE_DIR = os.path.abspath(os.getcwd())

# Caminhos utilizados no notebook (padronizados)
TRAIN_RAW = os.path.join(BASE_DIR, 'data', 'raw', 'twitter_training.csv')
VAL_RAW = os.path.join(BASE_DIR, 'data', 'raw', 'twitter_validation.csv')
VIS_DIR = os.path.join(BASE_DIR, 'reports', 'visualizacoes')
METRICS_DIR = os.path.join(BASE_DIR, 'reports', 'metrics')
MODEL_PATH = os.path.join(BASE_DIR, 'src', 'models', 'sentiment_model.pkl')
os.makedirs(VIS_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

print('[OK] Config inicial pronta — BASE_DIR:', BASE_DIR)

---
## 2. Análise Exploratória (EDA) <a id='eda'></a>

Nesta seção: carregar os CSVs brutos, inspecionar tamanhos, e salvar plots (gráficos) resumidos.

In [ ]:
# EDA: carregar dados e gerar graficos
# Definimos nomes de colunas esperados (os CSVs originais podem nao conter header)
cols = ['tweet_id','entity','sentiment','text']
# Verificacao de existencia dos arquivos brutos - evita falhas silenciosas
if not os.path.exists(TRAIN_RAW) or not os.path.exists(VAL_RAW):
    raise FileNotFoundError(f'Esperado arquivos em {TRAIN_RAW} e {VAL_RAW} — coloque os CSVs em data/raw/')
# Leitura com parametros robustos (encoding e engine)
df_train = pd.read_csv(TRAIN_RAW, names=cols, header=None, engine='python', encoding='utf-8')
df_val = pd.read_csv(VAL_RAW, names=cols, header=None, engine='python', encoding='utf-8')
# Marcamos o split para uniao posterior se necessario
df_train['split'] = 'train'
df_val['split'] = 'validation'
df = pd.concat([df_train, df_val], ignore_index=True)
print(f'Dados carregados: train={len(df_train)} | validation={len(df_val)} | total={len(df)}')
# Calculamos o comprimento dos textos (em palavras) para inspecionar a distribuicao
df['text_length'] = df['text'].astype(str).apply(lambda s: len(s.split()))
plt.figure(figsize=(10,5))
sns.histplot(df['text_length'], bins=40, kde=True)
plt.title('Distribuicao de comprimento de texto')
plt.xlabel('Numero de palavras')
plt.tight_layout()
plt.savefig(os.path.join(VIS_DIR, 'text_length.png'))
plt.close()
# Top words: conteudo bruto (sem limpeza) — util para entender ruído e tokens frequentes
all_words = ' '.join(df['text'].astype(str)).lower().split()
top_raw = pd.Series(all_words).value_counts().head(20)
plt.figure(figsize=(12,5))
top_raw.plot(kind='bar')
plt.title('Top words (raw)')
plt.tight_layout()
plt.savefig(os.path.join(VIS_DIR, 'top_words_raw.png'))
plt.close()
print('[OK] EDA: graficos salvos em reports/visualizacoes')

---
## 3. Pré-processamento <a id='preprocessing'></a>

As funções abaixo realizam: limpeza básica (remoção de URLs/menções/caracteres), remoção de stopwords (palavras carregam pouco ou nenhum significado semântico relevante para a análise do conteúdo principal de um texto) em inglês, e lematização (palavras na sua forma base) usando POS-tagging (categoriza as palavras em suas respectivas classes gramaticais) para melhorar a normalização das palavras.

In [ ]:
# Imports específicos para NLP com explicacoes
from nltk.corpus import stopwords, wordnet  # stopwords para filtro; wordnet para mapear POS em lematizacao
from nltk.stem import WordNetLemmatizer  # lematizador baseado em WordNet
from nltk.tokenize import word_tokenize  # tokenizacao que lida com pontuacao/contracoes
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Limpeza: lowercase, remove urls, mentions, hashtags, pontuacao e digitos
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_stopwords_en(text):
    # Remove stopwords em ingles (usa tokenizacao do NLTK)
    if not isinstance(text, str):
        return ''
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text, language='english')
    filtered = [w for w in tokens if w.lower() not in stop_words]
    return ' '.join(filtered)

def lemmatize_text_en(text):
    # Lematizacao com POS para melhor precisão (adjetivos, verbos, substantivos, advérbios)
    if not isinstance(text, str):
        return ''
    tokens = word_tokenize(text, language='english')
    try:
        pos_tags = nltk.pos_tag(tokens)
    except Exception:
        pos_tags = [(t, '') for t in tokens]
    def _get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        if tag.startswith('V'):
            return wordnet.VERB
        if tag.startswith('N'):
            return wordnet.NOUN
        if tag.startswith('R'):
            return wordnet.ADV
        return wordnet.NOUN
    lemmas = []
    for token, tag in pos_tags:
        wn_tag = _get_wordnet_pos(tag) if tag else wordnet.NOUN
        lemmas.append(lemmatizer.lemmatize(token, wn_tag))
    return ' '.join(lemmas)

# Aplicacao das funcoes aos datasets (cria colunas intermediarias)
df_train_proc = df_train.copy()
df_val_proc = df_val.copy()
df_train_proc['text_clean'] = df_train_proc['text'].apply(clean_text)
df_train_proc['text_no_stop'] = df_train_proc['text_clean'].apply(remove_stopwords_en)
df_train_proc['text_lemmatized'] = df_train_proc['text_no_stop'].apply(lemmatize_text_en)
df_val_proc['text_clean'] = df_val_proc['text'].apply(clean_text)
df_val_proc['text_no_stop'] = df_val_proc['text_clean'].apply(remove_stopwords_en)
df_val_proc['text_lemmatized'] = df_val_proc['text_no_stop'].apply(lemmatize_text_en)
print('[OK] Pre-processamento aplicado (dados em memoria)')

---
## 4. Modelagem <a id='modeling'></a>

Treinamos 3 classificadores usando um pipeline TF-IDF (ajuda a identificar as palavras-chave mais importantes de um texto) + classificador. Cada passo tem comentário explicando a escolha de parâmetros e tratamento de outputs para métricas (incluindo tentativa de obter probabilidades/decision scores para ROC/PR).

In [ ]:
# Modelagem e avaliacao
from sklearn.pipeline import Pipeline  # organiza vetorizador+classificador em um unico objeto
from sklearn.feature_extraction.text import TfidfVectorizer  # converte textos em features numéricas ponderadas
from sklearn.linear_model import LogisticRegression  # baseline robusto para classificacao binaria/multiclasse
from sklearn.naive_bayes import MultinomialNB  # bom para texto esparso e rápido
from sklearn.svm import LinearSVC  # SVM linear eficiente para grandes features de texto
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score  # metricas e curvas

# Preparamos X/y e removemos textos vazios — evita erros no vetorizador
X_train = df_train_proc['text_lemmatized'].astype(str)
y_train = df_train_proc['sentiment']
X_val = df_val_proc['text_lemmatized'].astype(str)
y_val = df_val_proc['sentiment']
mask_train = X_train.str.strip().replace('', np.nan).notna()
mask_val = X_val.str.strip().replace('', np.nan).notna()
X_train = X_train[mask_train]; y_train = y_train[mask_train]
X_val = X_val[mask_val]; y_val = y_val[mask_val]
print(f'Treino: {len(X_train)} | Validation: {len(X_val)}')

# Dicionario de modelos: facilidade para loop e comparacao
models = {
    'LogisticRegression': LogisticRegression(max_iter=2000, random_state=42),
    'MultinomialNB': MultinomialNB(),
    'LinearSVC': LinearSVC(max_iter=20000, random_state=42)
}
results = {}
for name, clf in models.items():
    print(f'[MODEL] Treinando {name}...')
    pipe = Pipeline([('tfidf', TfidfVectorizer(max_features=15000, ngram_range=(1,2))), ('clf', clf)])
    t0 = time.time(); pipe.fit(X_train, y_train); t1 = time.time(); train_time = t1 - t0
    t0p = time.time(); preds = pipe.predict(X_val); t1p = time.time(); predict_time = t1p - t0p
    acc = accuracy_score(y_val, preds); f1 = f1_score(y_val, preds, average='macro')
    report = classification_report(y_val, preds, output_dict=True)
    cm = confusion_matrix(y_val, preds)
    # Tentamos obter scores para ROC/PR: preferimos predict_proba (retorna a probabilidade de que uma amostra pertença a cada uma das classes disponíveis), senão usamos decision_function (indica a margem de confiança do modelo; quanto maior o valor absoluto, mais confiante o modelo está na sua classificação) quando disponivel
    y_score = None
    try:
        y_score = pipe.predict_proba(X_val)
    except Exception:
        try:
            decision = pipe.decision_function(X_val)
            if decision.ndim == 1:
                # transformar vetor 1D em array 2-colunas (negativo,positivo) para compatibilidade
                decision = np.vstack([-decision, decision]).T
            y_score = decision
        except Exception:
            y_score = None
    results[name] = {
        'pipeline': pipe, 'accuracy': acc, 'f1_macro': f1, 'train_time_seconds': train_time, 'predict_time_seconds': predict_time,
        'report': report, 'confusion_matrix': cm.tolist(), 'y_score': y_score
    }
    print(f'[RESULT] {name} — Accuracy: {acc:.4f} | F1-macro: {f1:.4f}')

# Escolher melhor por F1-macro (avalia o desempenho de modelos de classificação multiclasse) e salvar o pipeline final em disco (joblib)
best = max(results.keys(), key=lambda k: results[k]['f1_macro'])
best_pipeline = results[best]['pipeline']
joblib.dump(best_pipeline, MODEL_PATH)
print(f'[OK] Melhor modelo: {best} salvo em: {MODEL_PATH}')

# Salvar metricas resumidas em JSON (util para dashboards/relatorios)
metrics_out = {'best_model': best, 'results': {}}
for k in results:
    metrics_out['results'][k] = {
        'accuracy': results[k]['accuracy'],
        'f1_macro': results[k]['f1_macro'],
        'train_time_seconds': results[k]['train_time_seconds'],
        'predict_time_seconds': results[k]['predict_time_seconds'],
        'classification_report': results[k]['report'],
        'confusion_matrix': results[k]['confusion_matrix']
    }
with open(os.path.join(METRICS_DIR, 'model_metrics.json'), 'w') as f:
    json.dump(metrics_out, f, indent=2)
print('[OK] Metricas salvas em reports/metrics/model_metrics.json')

---
## 5. Visualizações Comparativas (ROC / PR / Confusion) <a id='visualizations'></a>

Nesta seção geramos um ROC comparativo, um PR comparativo e um arquivo com todas as matrizes de confusão lado-a-lado. Cada bloco tenta usar `y_score` salvo nos resultados — se não houver score (por exemplo, MultinomialNB tem predict_proba, SVC pode não ter), os plots são pulados para esse modelo. A curva ROC (Receiver Operating Characteristic) mostra a relação entre taxa de verdadeiros positivos (coordenada y) e taxa de falsos positivos (coordenada x) ao variar o limiar — útil para comparar separabilidade do modelo; a curva Precision‑Recall (PR) mostra a troca de desempenho entre precisão e recall e costuma ser mais informativa quando as classes estão desbalanceadas (foca em quão bem o modelo encontra a classe positiva sem gerar muitos falsos positivos).

In [ ]:
# Geracao de graficos comparativos
classes_all = np.unique(y_val)
try:
    from sklearn.preprocessing import label_binarize
    y_val_b_all = label_binarize(y_val, classes=classes_all)
except Exception:
    y_val_b_all = None

# ROC comparativo — agregamos curvas quando temos scores multi-classe binarizados
plt.figure(figsize=(8,6))
plotted_any = False
for name in results:
    y_score = results[name].get('y_score')
    if y_score is None or y_val_b_all is None:
        continue
    try:
        fpr, tpr, _ = roc_curve(y_val_b_all.ravel(), y_score.ravel())
        auc_val = None
        try:
            auc_val = roc_auc_score(y_val_b_all, y_score, average='macro', multi_class='ovr')
        except Exception:
            auc_val = None
        label = f'{name}'
        if auc_val is not None:
            label += f' (AUC={auc_val:.3f})'
        plt.plot(fpr, tpr, lw=2, label=label)
        plotted_any = True
    except Exception:
        continue
if plotted_any:
    plt.plot([0,1],[0,1],'k--', linewidth=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title('Comparative ROC Curves (all models)')
    plt.legend(loc='lower right'); plt.tight_layout()
    plt.savefig(os.path.join(VIS_DIR, 'comparison_roc.png'))
    plt.show()
    print('[OK] ROC comparativo salvo')
else:
    print('[WARN] Nenhum score disponivel para plotting ROC comparativo')

# PR comparativo
plt.figure(figsize=(8,6))
plotted_any = False
for name in results:
    y_score = results[name].get('y_score')
    if y_score is None or y_val_b_all is None:
        continue
    try:
        precision, recall, _ = precision_recall_curve(y_val_b_all.ravel(), y_score.ravel())
        ap = None
        try:
            ap = average_precision_score(y_val_b_all, y_score, average='macro')
        except Exception:
            ap = None
        label = f'{name}'
        if ap is not None:
            label += f' (AP={ap:.3f})'
        plt.plot(recall, precision, lw=2, label=label)
        plotted_any = True
    except Exception:
        continue
if plotted_any:
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title('Comparative Precision-Recall Curves (all models)')
    plt.legend(loc='lower left'); plt.tight_layout()
    plt.savefig(os.path.join(VIS_DIR, 'comparison_pr.png'))
    plt.close()
    print('[OK] PR comparativo salvo')
else:
    print('[WARN] Nenhum score disponivel para plotting PR comparativo')

# Confusion matrices lado-a-lado (sempre disponiveis a partir das preds)
model_names = list(results.keys())
cms = [np.array(results[nm]['confusion_matrix']) for nm in model_names]
if len(cms) > 0:
    vmax = max(cm.max() for cm in cms)
    fig, axes = plt.subplots(1, len(model_names), figsize=(6 * len(model_names), 5))
    if len(model_names) == 1:
        axes = [axes]
    for ax, nm, cm in zip(axes, model_names, cms):
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes_all, yticklabels=classes_all, vmin=0, vmax=vmax, ax=ax)
        ax.set_title(f'Confusion — {nm}'); ax.set_xlabel('Predito'); ax.set_ylabel('Real')
    plt.tight_layout(); plt.savefig(os.path.join(VIS_DIR, 'comparison_confusion_matrices.png'))
    plt.close(); print('[OK] Matrizes de confusao comparativas salvas')
else:
    print('[WARN] Nenhuma matriz de confusao disponivel')

---
## 6. Deploy com Docker (Opcional) <a id='deploy'></a>

Gera um `Dockerfile` de exemplo em `src/api/Dockerfile` para empacotar a aplicação. Comentário no código explica cada instrução.

In [ ]:
# Gerar Dockerfile de exemplo (opcional)
# O Dockerfile básico instala dependências e expõe a porta do servidor
dockerfile_content = '''
FROM python:3.9-slim
WORKDIR /app
COPY requirements.txt .  # copia manifest de dependencias
RUN pip install --no-cache-dir -r requirements.txt  # instala dependencias no container
COPY . .  # copia o codigo da aplicacao
EXPOSE 8000  # expõe porta padrao da app
CMD ["python", "manage.py", "runserver", "0.0.0.0:8000"]
'''
os.makedirs(os.path.join(BASE_DIR, 'src', 'api'), exist_ok=True)
with open(os.path.join(BASE_DIR, 'src', 'api', 'Dockerfile'), 'w') as f:
    f.write(dockerfile_content)
print('[OK] Dockerfile criado (src/api/Dockerfile)')